In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [3]:
data = pd.read_csv("Train.csv")

In [4]:
data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,3.0,0.0,"O'Donoghue, Ms. Bridget",female,NaN,0.0,0.0,364856,7.750,NaN,Q,NaN,NaN,NaN
1,2.0,0.0,"Morley, Mr. Henry Samuel (""Mr Henry Marshall"")",male,39.0,0.0,0.0,250655,26.000,NaN,S,NaN,NaN,NaN
2,2.0,1.0,"Smith, Miss. Marion Elsie",female,40.0,0.0,0.0,31418,13.000,NaN,S,9,NaN,NaN
3,3.0,1.0,"Goldsmith, Mrs. Frank John (Emily Alice Brown)",female,31.0,1.0,1.0,363291,20.525,NaN,S,C D,NaN,"Strood, Kent, England Detroit, MI"
4,3.0,1.0,"McCoy, Miss. Agnes",female,NaN,2.0,0.0,367226,23.250,NaN,Q,16,NaN,NaN


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 14 columns):
pclass       1009 non-null float64
survived     1009 non-null float64
name         1009 non-null object
sex          1009 non-null object
age          812 non-null float64
sibsp        1009 non-null float64
parch        1009 non-null float64
ticket       1009 non-null object
fare         1008 non-null float64
cabin        229 non-null object
embarked     1008 non-null object
boat         374 non-null object
body         98 non-null float64
home.dest    582 non-null object
dtypes: float64(7), object(7)
memory usage: 110.4+ KB


In [6]:
data.age.fillna(np.mean(data.age), inplace = True)

In [7]:
le = LabelEncoder()

In [8]:
le.fit(data.sex)

LabelEncoder()

In [9]:
data["sex"] = le.transform(data.sex)

In [10]:
data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,3.0,0.0,"O'Donoghue, Ms. Bridget",0,29.838978,0.0,0.0,364856,7.750,NaN,Q,NaN,NaN,NaN
1,2.0,0.0,"Morley, Mr. Henry Samuel (""Mr Henry Marshall"")",1,39.000000,0.0,0.0,250655,26.000,NaN,S,NaN,NaN,NaN
2,2.0,1.0,"Smith, Miss. Marion Elsie",0,40.000000,0.0,0.0,31418,13.000,NaN,S,9,NaN,NaN
3,3.0,1.0,"Goldsmith, Mrs. Frank John (Emily Alice Brown)",0,31.000000,1.0,1.0,363291,20.525,NaN,S,C D,NaN,"Strood, Kent, England Detroit, MI"
4,3.0,1.0,"McCoy, Miss. Agnes",0,29.838978,2.0,0.0,367226,23.250,NaN,Q,16,NaN,NaN


In [11]:
col = ["pclass" , "sex" , "age" , "sibsp" , "parch" ]

In [36]:
X = data.loc[:,["pclass" , "sex" , "age" , "sibsp" , "parch" ]]

In [37]:
col_2 =["survived"]

In [38]:
y = data["survived"]

In [39]:
def entropy(column):
    vals , counts = np.unique(column , return_counts = True)
    total = len(column)
    acc = 0
    for i in counts:
        acc += (i/total) * np.log2(i/total)
    return -acc

In [40]:
entropy(y)

0.9570419991692567

In [41]:
def info_gain(X , y , label):
    selected = np.mean(X[label])
    y_left = y[X[label]<selected]
    y_right = y[X[label]>= selected]
    
    if len(y_left) == 0 or len(y_right) == 0:
        return -10000
    left = (len(y_left)/len(y)) * entropy(y_left)
    right = (len(y_right)/len(y)) * entropy(y_right)
    
    return entropy(y)-left-right

In [42]:
for label in X.columns:
    print((label,info_gain(X , y , label)))

('pclass', 0.055456910002982474)
('sex', 0.19274737190850932)
('age', 0.0010525742338488575)
('sibsp', 0.006492394392888956)
('parch', 0.019756080122948216)


In [43]:
class Node:
    def __init__(self, label = None , value = None , result = None):
        self.label = label
        self.value = value
        self.result = result

In [44]:
class CustomDecisionTree:
    def __init__(self , max_depth = 5):
        self.max_depth = max_depth
        
    def fit(self, X,y):
        
        self.root = self.generate(X , y , self.max_depth)
        
    def generate(self, X , y , depth):
        
        if depth == 1:
            return Node(result = np.mean(y))
        gain = []
        for label in X.columns:
            gain.append((info_gain(X , y , label),label))
                        
        sel = max(gain)[1]
                        
        pivot = np.mean(X[sel])
                        
        left = X[sel]<pivot
        right = X[sel]>= pivot
        X_left, y_left = X[left], y[left]
        X_right, y_right = X[right], y[right]
        
        node = Node(sel,pivot)
        Node.left = self.generate(X_left , y_left , depth-1)
        Node.right = self.generate(X_right , y_right , depth-1)
    
        return node
    def predict(self , X ):
        y=[]
        for index , rows in X.iterrows():
            res = int(self.predict_point(rows , self.root)>.5)
        return np.array(y)
    def predict_point(self , row , node):
        if node.label == None:
            return node.result
        
        if row[node.label] < node.value:
            return self.predict_point(row, node.left)
        else:
            return self.predict_point(row, node.right)
    def score(self,X,y):
        yp = self.predict(X)
        return np.sum(yp.values==y) /len(y)
        

In [45]:
model = CustomDecisionTree()

In [46]:
model.fit(X,y)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

In [32]:
model.predict(X_test)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,


In [77]:
D.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 13 columns):
pclass       300 non-null float64
name         300 non-null object
sex          300 non-null object
age          234 non-null float64
sibsp        300 non-null float64
parch        300 non-null float64
ticket       300 non-null object
fare         300 non-null float64
cabin        66 non-null object
embarked     299 non-null object
boat         112 non-null object
body         23 non-null float64
home.dest    163 non-null object
dtypes: float64(6), object(7)
memory usage: 22.3+ KB


In [78]:
y1 = D.loc[:,["pclass" , "sex" , "age" , "sibsp" , "parch" ]]

In [79]:
y1["sex"] = le.transform(y1.sex)

In [80]:
y1.age.fillna(np.mean(y1.age), inplace = True) 

In [135]:
output = m1.predict(y1)

In [142]:
from sklearn.ensemble import RandomForestClassifier

In [192]:
m2 = RandomForestClassifier(max_depth=5)

In [193]:
m2.fit(X , y)

c:\users\shivang\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [194]:
output2 = m2.predict(y1)

In [195]:
output2 = {'Id': range(len(output2)),
              'survived': output2}

In [196]:
op2 = pd.DataFrame.from_dict(output2)

In [197]:
op2.to_csv("foo2.csv",index = False , index_label = "Id")

In [191]:
m3 = RandomForestClassifier(max_depth=20)

In [182]:
m3.fit(X_train,y_train)

c:\users\shivang\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=20, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [183]:
m3.predict(X_test)

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0.,
       1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0.,
       1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1.,
       0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1.,
       0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1.,
       1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 1.

In [184]:
m3.score(X_test,y_test)

0.7627627627627628

# Using Neural Networks

In [19]:
from keras.layers import *
from keras.models import Model

In [25]:
from sklearn.preprocessing import OneHotEncoder

In [56]:
ohe = OneHotEncoder()
y_ = (y.values).reshape(-1,1)
y_ohe = ohe.fit_transform(y_)
X_ = X.values

c:\users\shivang\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [68]:
in_layer = Input(shape = [5])
dense_1 = Dense(500 , activation = "relu")(in_layer)

dense_2 = Dense(300 , activation = "relu")(dense_1)

dense_3 = Dense(10 , activation = "relu")(dense_2)

out2 = Dense(2 , activation = "softmax")(dense_3)

In [69]:
model = Model(inputs = [in_layer], outputs = [out2])

In [70]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 5)                 0         
_________________________________________________________________
dense_24 (Dense)             (None, 500)               3000      
_________________________________________________________________
dense_25 (Dense)             (None, 300)               150300    
_________________________________________________________________
dense_26 (Dense)             (None, 10)                3010      
_________________________________________________________________
dense_27 (Dense)             (None, 2)                 22        
Total params: 156,332
Trainable params: 156,332
Non-trainable params: 0
_________________________________________________________________


In [71]:
model.compile(optimizer = "adam" , loss = "categorical_crossentropy" , metrics = ["acc"])
hist = model.fit(X_ , y_ohe , epochs = 100 , batch_size = 100)

Epoch 1/100
1009/1009 [==============================] - 1s 626us/step - loss: 0.7486 - acc: 0.5719
Epoch 2/100
1009/1009 [==============================] - 0s 138us/step - loss: 0.6552 - acc: 0.6303
Epoch 3/100
1009/1009 [==============================] - 0s 119us/step - loss: 0.6201 - acc: 0.6511
Epoch 4/100
1009/1009 [==============================] - 0s 128us/step - loss: 0.5759 - acc: 0.7027
Epoch 5/100
1009/1009 [==============================] - 0s 145us/step - loss: 0.5889 - acc: 0.7007
Epoch 6/100
1009/1009 [==============================] - 0s 116us/step - loss: 0.5973 - acc: 0.6967
Epoch 7/100
1009/1009 [==============================] - 0s 103us/step - loss: 0.5420 - acc: 0.7195
Epoch 8/100
1009/1009 [==============================] - 0s 100us/step - loss: 0.5760 - acc: 0.7146
Epoch 9/100
1009/1009 [==============================] - 0s 101us/step - loss: 0.5536 - acc: 0.7433
Epoch 10/100
1009/1009 [==============================] - 0s 94us/step - loss: 0.4983 - acc: 0.7810


1009/1009 [==============================] - 0s 81us/step - loss: 0.4189 - acc: 0.8097
Epoch 83/100
1009/1009 [==============================] - 0s 75us/step - loss: 0.4317 - acc: 0.7988
Epoch 84/100
1009/1009 [==============================] - 0s 97us/step - loss: 0.4238 - acc: 0.8117
Epoch 85/100
1009/1009 [==============================] - 0s 96us/step - loss: 0.4248 - acc: 0.8038
Epoch 86/100
1009/1009 [==============================] - 0s 102us/step - loss: 0.4347 - acc: 0.8038
Epoch 87/100
1009/1009 [==============================] - 0s 81us/step - loss: 0.4207 - acc: 0.8057
Epoch 88/100
1009/1009 [==============================] - 0s 84us/step - loss: 0.4215 - acc: 0.8127
Epoch 89/100
1009/1009 [==============================] - 0s 97us/step - loss: 0.4367 - acc: 0.8028
Epoch 90/100
1009/1009 [==============================] - 0s 103us/step - loss: 0.4373 - acc: 0.8087
Epoch 91/100
1009/1009 [==============================] - 0s 89us/step - loss: 0.4255 - acc: 0.8167
Epoch 92/10